### 预处理：获取文件中的所有飞机名称

首先pandas读文件

In [16]:
import pandas as pd

original_file = pd.read_csv('D:\\Study\\Computing\\发射机与目标机判定\\data\\51st Bisons vs CNF Rd 1__1HZ.csv') # 读取csv文件
Ids = original_file['Id'].values # 读取Id列
names = [] # 创建空列表用于储存飞机名称

In [17]:
original_file.loc[original_file['Id'] == '102']['Type'].values[0]

'Air+FixedWing'

获取文件中的所有飞机名称

In [18]:
for Id in Ids:
    found = False
    type_name = original_file.loc[original_file['Id'] == Id]['Type'].values[0] # 读取Type列并获取第一个值
    if type_name.find('Air') == -1:
        # 查找所有带有“Air”的目标——即飞机，储存在names中
        # 去除导弹
        continue
    for name in names:
        # 去除重复的目标
        if Id == name:
            found = True
            break
    if not found:
        names.append(Id)


print(names)

['102', '202', '106', '10A', '109', '107', '108', '10C', '105', '10B', '104', '10E', '10F', '110']


## 现在开始处理问题

首先还是读文件

In [19]:
original_filename = '51st vs 36th R1__1HZ.csv'

ttmp = 'D:\\Study\\Computing\\发射机与目标机判定\\data\\' + original_filename # 读取csv文件

original_file = pd.read_csv(ttmp)
Ids = original_file['Id'].values
names = []

获取空战发生的时间区间

In [20]:
time_min = original_file['Unix time'].min() # 获取最小时间
time_max = original_file['Unix time'].max() # 获取最大时间

In [21]:
time_file_name = original_filename + '_time' + '.txt' # 时间文件名

df = pd.DataFrame(index=range(time_max - time_min + 30)) # 根据指定时间区间创建空DataFrame

和之前一样获取文件中的所有飞机名称

In [22]:
for Id in Ids:
    found = False
    type_name = original_file.loc[original_file['Id'] == Id]['Type'].values[0]
    if type_name.find('Air') == -1:
        continue
    for name in names:
        if Id == name:
            found = True
            break
    if not found:
        names.append(Id)

现在开始遍历names，依次分析各架飞机，首先对一架飞机而言，例如name = '102'

In [23]:
import numpy as np

name = '102' # 选择目标

acts = [] # 用于储存动作序列
file = original_file[original_file.Id == name] # 获取Id为name的目标的所有数据，是原数据集的子集

start_time = file.iloc[0, 1] - time_min # 该飞机开始被记录的时间

# 获取飞机状态
roll = file['Roll'].values
pitch = file['Pitch'].values
yaw = file['Yaw'].values
ground_distance = file['Altitude'].values # 此处使用的是海拔高度而不是距地面距离

首先需要定义各种简单动作

In [24]:
from sklearn.linear_model import LinearRegression


def calc_b(y): 
    # R^2检验，返回值包括拟合斜率和线性拟合度，线性拟合度大于0.95则认为是线性关系，此时滑动窗口的长度不需要再减小
    reg = LinearRegression()
    len_ = len(y)
    y = y.reshape(-1, 1)
    x = [i for i in range(1, len_ + 1)]
    x = np.array(x)
    x = x.reshape(-1, 1)
    reg.fit(x, y)
    return reg.coef_, reg.score(x, y)


def decide_basic_action(yaw, pitch, roll, ground_dist):
    # 根据飞机状态判断基本动作

    # 计算各飞行参数的拟合斜率和线性拟合度
    yaw_b, yaw_r2 = calc_b(yaw)
    pitch_b, pitch_r2 = calc_b(pitch)
    roll_b, roll_r2 = calc_b(roll)
    ground_dist_b, ground_dist_r2 = calc_b(ground_dist)

    threshold = 0.95 # 线性拟合度阈值

    if yaw_r2 < threshold or pitch_r2 < threshold or roll_r2 < threshold or ground_dist_r2 < threshold:
        # 线性拟合度小于阈值，认为不是线性关系，返回0，需要减小滑动窗口长度
        return 0

    ans = [] # 储存基本动作

    if (yaw_b > -1) and (yaw_b < 1) and (ground_dist_b > -2) and (ground_dist_b < 2):
        ans.append(1) # 平飞：1
    if ground_dist_b > 2:
        ans.append(2) # 上升：2
    if ground_dist_b < -2:
        ans.append(3) # 下降：3
    if yaw_b > 1:
        ans.append(4) # 转弯：4
    if roll_b > 10:
        ans.append(5) # 翻滚：5
    if len(ans) == 0:
        return 0 # 无法检测，需要进一步缩小滑动窗口长度
    else:
        return ans

下面开始使用滑动窗口

In [25]:
end = len(roll) # 结束位置
cur = 0 # 当前位置
default_sliding_window_len = 10 # 默认滑动窗口长度

In [ ]:
acts = [] # 储存基本动作
while cur < end: # 遍历时间区间
    begin = cur # 更新当前位置
    sliding_window_len = default_sliding_window_len # 初始设置为默认滑动窗口长度

    act = decide_basic_action(
        yaw = yaw[begin : begin+sliding_window_len], 
        pitch = pitch[begin : begin+sliding_window_len],
        roll = roll[begin : begin+sliding_window_len],
        ground_dist = ground_distance[begin : begin+sliding_window_len]
        )
    # 基于当前滑动窗口长度判断基本动作
        
    while act == 0: 
        # 无法检测，需要进一步缩小滑动窗口长度直至可以检测
        sliding_window_len -= 1 # 缩小滑动窗口长度
        act = decide_basic_action(
            yaw=yaw[begin:begin + sliding_window_len],
            pitch=pitch[begin:begin + sliding_window_len],
            roll=roll[begin:begin + sliding_window_len],
            ground_dist=ground_distance[begin:begin + sliding_window_len]
            )
        # 基于当前滑动窗口长度再次判断基本动作
            
    for i in range(sliding_window_len):
        tmp = ''
        for a in act: # [1,2,3] -> '123'
            tmp += str(a) 
            # a = 1, 2, 3, 4, 5分别表示不同的基本动作
            # 例如tmp = '123'表示当前滑动窗口内的基本动作为平飞、上升、下降

        acts.append(tmp) # 记录基本动作序列，注意这是一个列表，列表中的每个元素都是一个字符串
        # 例如tmp = ['123' '123' '123']表示此次滑动窗口长度为3，窗口内的3个时刻基本动作序列依次为：
        # 1：平飞、上升、下降，2：平飞、上升、下降，3：平飞、上升、下降

    cur += sliding_window_len # 更新当前位置

    df = df.astype('object') # 用于储存字符串格式的基本动作序列
    
    df[name] = '0' # 初始化该列，统一设置成未起飞状态：0
    df.loc[start_time: start_time+len(acts)-1, name] = acts
    df.loc[start_time+len(acts):, name] = -1
    # 将acts嵌入df的一列，其中飞机起飞前设置为0，后续为基本动作序列，最后是占位的-1

现在使用循环处理所有飞机对象

In [ ]:
for name in names:
    acts = []
    file = original_file[original_file.Id == name]
    start_time = file.iloc[0, 1] - time_min
    roll = file['Roll'].values
    roll = np.array(roll)
    pitch = file['Pitch'].values
    pitch = np.array(pitch)
    yaw = file['Yaw'].values
    yaw = np.array(yaw)
    ground_distance = file['Altitude'].values
    ground_distance = np.array(ground_distance)

    start = 0
    end = len(roll)
    cur = 0
    default_sliding_window_len = 10

    while cur < end:
        begin = cur
        sliding_window_len = default_sliding_window_len
        act = decide_basic_action(
            yaw=yaw[begin:begin + sliding_window_len], 
            pitch=pitch[begin:begin + sliding_window_len],
            roll=roll[begin:begin + sliding_window_len],
            ground_dist=ground_distance[begin:begin + sliding_window_len]
            )
        
        while act == 0:
            sliding_window_len -= 1
            act = decide_basic_action(
                yaw=yaw[begin:begin + sliding_window_len],
                pitch=pitch[begin:begin + sliding_window_len],
                roll=roll[begin:begin + sliding_window_len],
                ground_dist=ground_distance[begin:begin + sliding_window_len]
                )
            
        for i in range(sliding_window_len):
            tmp = ''
            for a in act:
                tmp += str(a)
            acts.append(tmp)
        cur += sliding_window_len

        df = df.astype('object')
        df[name] = '0'
        df.loc[start_time: start_time+len(acts)-1, name] = acts #! -1
        df.loc[start_time+len(acts):, name] = -1

In [30]:
print(df)
tttmp = 'ques2_' + original_filename
df.to_csv(tttmp) # 将基本动作序列输出为csv文件

     102 202 104 105 107 106 10A 109 10D 108 10E 10C 10B 10F
0      3   3   0   0   0   0   0   0   0   0   0   0   0   0
1      3   3   0   0   0   0   0   0   0   0   0   0   0   0
2      3   3   0   0   0   0   0   0   0   0   0   0   0   0
3      3   3   0   0   0   0   0   0   0   0   0   0   0   0
4      1   1   0   0   0   0   0   0   0   0   0   0   0   0
...   ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..  ..
5174  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
5175  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
5176  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
5177  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
5178  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1

[5179 rows x 14 columns]


现在开始考虑由基本动作组成的复杂机动动作

In [31]:
def calc_yaw(yaw):
    len_ = len(yaw)
    sum = 0
    for i in range(1, len_):
        if np.abs(yaw[i] - yaw[i - 1]) > 300:
            sum += 5
        else:
            sum += np.abs(yaw[i] - yaw[i - 1])
            
    return sum

In [32]:
def decide_action(yaw, pitch, roll, altitude, actions):
    len_ = len(yaw)
    acts = [0, 0, 0, 0, 0] # 储存基本动作，分别为：平飞、上升、下降、转弯、翻滚，出现为1，否则为0
    
    # 统计区间内基本动作
    for a in actions:
        while a >= 1:
            acts[int(a % 10) - 1] += 1
            a = int(a / 10)

    # 判定定常盘旋
    threshold = 0.5
    height_threshold = 20
    roll_threshold = 10
    yaw_threshold = 10

    if acts[3] > 0 and calc_yaw(yaw=yaw) > 360 * threshold and np.max(altitude) - np.min(altitude) < height_threshold and np.min(np.abs(roll)) > roll_threshold:
        return 1
    
    # 判定急盘降
    altitude_b, _ = calc_b(altitude)
    if acts[3] > 0 and acts[2] > 0 and calc_yaw(yaw=yaw) > 360 * threshold and np.max(altitude) - np.min(altitude) > height_threshold and altitude_b < 0:
        return 2
    
    # 判定急盘升
    if acts[3] > 0 and acts[1] > 0 and calc_yaw(yaw=yaw) > 360 * threshold and np.max(altitude) - np.min(altitude) > height_threshold and altitude_b > 0:
        return 3
    
    # 判定俯冲
    if acts[2] > 0 and calc_yaw(yaw) < yaw_threshold and np.max(altitude) - np.min(altitude) > height_threshold and altitude_b < 0:
        return 4
    
    # 判定急拉起
    if acts[1] > 0 and calc_yaw(yaw) < yaw_threshold and np.max(altitude) - np.min(altitude) > height_threshold and altitude_b > 0:
        return 5
    
    # 判定半斤斗
    if acts[1] + acts[2] > 0 and np.max(pitch) - np.min(pitch) > 180 * threshold and np.max(roll) - np.min(roll) < roll_threshold:
        return 6
    
    # 判定半滚倒转
    if acts[1] + acts[2] > 0 and np.max(pitch) - np.min(pitch) > 180 * threshold and np.max(roll) - np.min(roll) > 180 * threshold:
        return 7
    
    # 判定滚筒
    if acts[4] > 0 and np.max(altitude) - np.min(altitude) < height_threshold and calc_yaw(yaw) < yaw_threshold and np.max(roll) - np.min(roll) > 180 *threshold:
        return 8
    
    # 判定战术转弯
    yaw_slope_threshold = 5
    yaw_b, _ = calc_b(yaw)
    if acts[3] > 0 and yaw_b > yaw_slope_threshold and calc_yaw(yaw) < 90 *threshold:
        return 9
    
    # 判定规避急转弯
    if acts[3] > 0 and yaw_b > yaw_slope_threshold and calc_yaw(yaw) > 90 *threshold:
        return 10
    
    # 判定低速 yoyo
    if acts[1] + acts[4] > 0 and altitude_b > 0 and np.max(roll) - np.min(roll) > 180 *threshold:
        return 12
    
    # 判定高速 yoyo
    if acts[2] + acts[4] > 0 and altitude_b < 0 and np.max(roll) - np.min(roll) > 180 *threshold:
        return 11
    
    return np.argmax(acts) + 13

| 平飞 | 上升 | 下降 | 转弯 | 翻滚 | 定常盘旋 | 急盘降 | 急盘升 | 俯冲 | 急拉起 | 半斤斗 | 半滚倒转 | 滚筒 | 战术转弯 | 规避急转弯 | 高速 yoyo | 低速 yoyo |
| --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 13 | 14 | 15 | 16 | 17 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 | 12 |

读入文件以及预处理

In [33]:
name = '51st vs 36th R1__1HZ.csv'
file_name = 'D:\\Study\\Computing\\发射机与目标机判定\\ques2_' + name
origin_name = 'D:\\Study\\Computing\\发射机与目标机判定\\data\\' + name
file = pd.read_csv(file_name)
origin = pd.read_csv(origin_name)

YAW = origin['Yaw'].values
ROLL = origin['Roll'].values
PITCH = origin['Pitch'].values
ALTITUDE = origin['Altitude'].values

time_min = origin['Unix time'].min()
time_max = origin['Unix time'].max()
df = pd.DataFrame(index=range(time_max - time_min + 30))
df = df.astype('object')

基本动作序列进行划分

In [34]:
def decompose(a): # 将一个整数分解为各个位上的数字：123 -> [3, 2, 1]
    ans = []
    while a >= 1:
        ans.append(int(a % 10))
        a = int(a / 10)
    return ans

def calc_b(y): 
    # R^2检验，返回值包括拟合斜率和线性拟合度，线性拟合度大于0.95则认为是线性关系，此时滑动窗口的长度不需要再减小
    reg = LinearRegression()
    len_ = len(y)
    y = y.reshape(-1, 1)
    x = [i for i in range(1, len_ + 1)]
    x = np.array(x)
    x = x.reshape(-1, 1)
    reg.fit(x, y)
    return reg.coef_, reg.score(x, y)

def calc_similarity(a, b):
    # 判断两个整数的各个位上的数字是否相同
    a_digits = decompose(a)
    b_digits = decompose(b)

    all_in_a = True # a中的数字是否都在b中

    for d in a_digits:
        if d not in b_digits:
            all_in_a = False
            break

    all_in_b = True # b中的数字是否都在a中

    for d in b_digits:
        if d not in a_digits:
            all_in_b = False
            break

    return all_in_a or all_in_b # Ture表示其中一个整数的各个位上的数字都在另一个整数中
    
def solve_3_1(data):# 对基本动作序列进行划分，每一段组成一个复杂动作
    start = 0
    while data[start] == 0:
        start += 1 # 找到第一个不为0的位置

    end = len(data)
    intervals = []
    sss = True

    while start < end - 1 and sss:
        cur = start
        flag = True
        while flag:
            if data[cur] == -1: # 到达基本序列的末尾（-1用作动作序列末尾的占位，也表示坠毁、降落）
                sss = False
                break

            if data[cur + 1] == -1:
                sss = False
                break

            flag = calc_similarity(data[cur], data[cur + 1]) 
            # Ture代表两个时刻的基本动作未发生突变，即只出现基本动作的增加或减少:123->12，而没有基本动作的改变:12->23
            cur += 1 # 切片长度加1直至基本动作发生突变

        interval = str(start) + '-' + str(cur - 1)
        # print(interval)
        intervals.append(interval) # 保存本次划分的动作片段位置
        start = cur

    return intervals

对之前已经得到的基本动作序列划分后进行识别

In [ ]:
flag = 0

for ids in list(file.columns.values):
    flag += 1
    if flag == 1:
        continue
    data = file[ids].values # 指定飞机的基本动作序列，
    # .values将DataFrame转换为ndarray，其中元素类型从object转换为int
    intervals = solve_3_1(data) # 将基本动作序列划分为一串子序列，每一段子序列组成一个复杂动作
    actions = [] # 保存研究区间内的动作序列

    for interval in intervals:
        start, end = interval.split('-') # interval的格式为'起始时刻-终止时刻'
        start, end = int(start), int(end) # 将起始时刻和终止时刻转换为整数

        # 通过判断区间内的基本动作序列，确定区间内的复杂动作
        action = decide_action(
            yaw=YAW[start:end+1],
            roll=ROLL[start:end+1],
            pitch=PITCH[start:end+1],
            altitude=ALTITUDE[start:end+1],
            actions=data[start:end+1])
        
        for i in range(end - start + 1):
            actions.append(action) # 将复杂动作序列添加到动作序列中

    df[ids] = '0'
    ff = origin[origin.Id == ids]
    start_time = ff.iloc[0, 1] - time_min

    df.loc[start_time: start_time+len(actions)-1, name] = actions #! -1
    df.loc[start_time+len(actions):, name] = -1

输出结果

In [36]:
tttmp = 'ques3_1_' + name
df.to_csv(tttmp)